In [3]:
import numpy as np
import pandas as pd
import pycountry as pc
import pycountry_convert as pyc
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

# Carga de Datos

In [6]:
spotify_data = pd.read_csv('universal_top_spotify_songs.csv',delimiter = ",")
spotify_data.head()

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",1,0,0,NaN,2025-03-02,96,False,...,6,-7.777,0,0.0304,0.3080,0.0000,0.122,0.535,157.969,3
1,4wJ5Qq0jBN4ajy7ouZIV1c,APT.,"ROSÉ, Bruno Mars",2,0,0,NaN,2025-03-02,87,False,...,0,-4.477,0,0.2600,0.0283,0.0000,0.355,0.939,149.027,4
2,2CGNAOSuO1MEFCbBRgUzjd,luther (with sza),"Kendrick Lamar, SZA",3,0,0,NaN,2025-03-02,89,False,...,2,-7.546,1,0.1250,0.2510,0.0000,0.248,0.576,138.008,4
3,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,Kendrick Lamar,4,0,0,NaN,2025-03-02,92,True,...,1,-7.001,1,0.0776,0.0107,0.0000,0.141,0.214,101.061,4
4,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,5,0,0,NaN,2025-03-02,94,False,...,2,-10.171,1,0.0358,0.2000,0.0608,0.117,0.438,104.978,4


# Limpieza de Datos

In [18]:
spotify_data.shape

(1775230, 25)

In [20]:
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1775230 entries, 0 to 1775229
Data columns (total 25 columns):
 #   Column              Dtype  
---  ------              -----  
 0   spotify_id          object 
 1   name                object 
 2   artists             object 
 3   daily_rank          int64  
 4   daily_movement      int64  
 5   weekly_movement     int64  
 6   country             object 
 7   snapshot_date       object 
 8   popularity          int64  
 9   is_explicit         bool   
 10  duration_ms         int64  
 11  album_name          object 
 12  album_release_date  object 
 13  danceability        float64
 14  energy              float64
 15  key                 int64  
 16  loudness            float64
 17  mode                int64  
 18  speechiness         float64
 19  acousticness        float64
 20  instrumentalness    float64
 21  liveness            float64
 22  valence             float64
 23  tempo               float64
 24  time_signature      int6

Descripción de las columnas:

1. spotify_id: Identificador único de cada canción en Spotify.
2. name: Nombre de la canción.
3. artists: Nombre del artista o artistas.
4. daily_rank: Posición de la canción en el ranking diario.
5. daily_movement: Movimiento diario en el ranking (probablemente un valor numérico que indica cuántas posiciones subió o bajó).
6. weekly_movement: Movimiento semanal en el ranking.
7. country: País asociado al ranking.
8. snapshot_date: Fecha en la que se tomó la métrica.
9. popularity: Popularidad de la canción (un valor entre 0 y 100 siendo 100 la mas popular).
10. is_explicit: Indica si la canción tiene contenido explícito (True/False).
11. duration_ms: Duración de la canción en milisegundos.
12. album_name: Nombre del álbum al que pertenece la canción.
13. album_release_date: Fecha de lanzamiento del álbum.
14. danceability: Medida de qué tan bailable es la canción (0 a 1). A value of 0.0 is least danceable and 1.0 is most danceable.
15. energy: Medida de la intensidad o energía de la canción (0 a 1). Energy is a measure from 0.0 to 1.0
16. key: Tonalidad musical de la canción (valores numéricos que representan notas musicales). Integers map to pitches using standard Pitch Class notation
17. loudness: Volumen promedio de la canción (en decibelios). Values typically range between -60 and 0 db.
18. mode: Modalidad de la canción (0 = menor, 1 = mayor). the type of scale from which its melodic content is derived
19. speechiness: Medida de la presencia de palabras habladas en la canción (0 a 1). Values between 0.33 and 0.66 describe tracks that may contain both music and speech 1 es talk show, audio book, poetry
20. acousticness: Medida de acústica en la canción (0 a 1). Range: 0 - 1 Example: 0.00242 1.0 represents high confidence the track is acoustic
21. instrumentalness: Medida de instrumentalidad en la canción (0 a 1). The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
22. liveness: Medida de la presencia de una audiencia en vivo (0 a 1). A value above 0.8 provides strong likelihood that the track is live.
23. valence: Medida de la positividad o felicidad de la canción (0 a 1). Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric),
24. tempo: Velocidad o ritmo de la canción (en BPM, beats por minuto). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
25. time_signature: Compás de la canción (por ejemplo, 4/4, 3/4). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4".

In [23]:
spotify_data.describe().T

,count,mean,std,min,25%,50%,75%,max
daily_rank,1775230.0,25.492264,14.428933,1.000000,13.0000,25.000000,38.000000,50.000
daily_movement,1775230.0,0.920829,6.990025,-49.000000,-1.0000,0.000000,2.000000,49.000
weekly_movement,1775230.0,2.772361,12.127969,-49.000000,-3.0000,0.000000,5.000000,49.000
popularity,1775230.0,75.998221,15.770881,0.000000,65.0000,80.000000,88.000000,100.000
duration_ms,1775230.0,193581.860801,49669.671663,0.000000,161831.0000,185933.000000,218423.000000,939666.000
danceability,1775230.0,0.678811,0.141580,0.000000,0.5840,0.701000,0.782000,0.988
energy,1775230.0,0.648794,0.167205,0.000020,0.5510,0.668000,0.765000,0.998
key,1775230.0,5.543401,3.580589,0.000000,2.0000,6.000000,9.000000,11.000
loudness,1775230.0,-6.652486,3.449654,-54.341000,-7.8050,-6.035000,-4.712000,3.233
mode,1775230.0,0.537784,0.498570,0.000000,0.0000,1.000000,1.000000,1.000


In [25]:
spotify_data.describe(include=[object]).T

,count,unique,top,freq
spotify_id,1775230,21084,6dOtVTDdiauQNBQEDOtlAB,13198
name,1775200,18553,BIRDS OF A FEATHER,13198
artists,1775201,11908,Billie Eilish,27480
country,1750973,72,DO,24525
snapshot_date,1775230,490,2024-03-16,3942
album_name,1774409,14057,HIT ME HARD AND SOFT,25707
album_release_date,1774572,2668,2024-05-17,29532


In [26]:
# Valores faltantes -> Nulos
print(f"% de Valores faltantes por columna: \n {((spotify_data.isnull().sum() / spotify_data.shape[0]) * 100).round(3)}")

% de Valores faltantes por columna: 
 spotify_id            0.000
name                  0.002
artists               0.002
daily_rank            0.000
daily_movement        0.000
weekly_movement       0.000
country               1.366
snapshot_date         0.000
popularity            0.000
is_explicit           0.000
duration_ms           0.000
album_name            0.046
album_release_date    0.037
danceability          0.000
energy                0.000
key                   0.000
loudness              0.000
mode                  0.000
speechiness           0.000
acousticness          0.000
instrumentalness      0.000
liveness              0.000
valence               0.000
tempo                 0.000
time_signature        0.000
dtype: float64


In [29]:
# Podemos ver que hay filas sin informacion de la cancion y revisando los IDs parece que son valores erroneos, 
# Borrmos los nulos en 'artists', 'album_name' y 'album_release_date'         
spotify_data.dropna(subset=['artists'], inplace=True)
spotify_data.dropna(subset=['album_name'], inplace=True)
spotify_data.dropna(subset=['album_release_date'], inplace=True)

In [23]:
# De acuerdo a la documentacion, completamos con 'Global' donde 'country' es nulo
spotify_data['country'] = spotify_data['country'].fillna('Global')

In [25]:
spotify_data.isnull().sum()

spotify_id            0
name                  0
artists               0
daily_rank            0
daily_movement        0
weekly_movement       0
country               0
snapshot_date         0
popularity            0
is_explicit           0
duration_ms           0
album_name            0
album_release_date    0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
time_signature        0
dtype: int64

In [27]:
# Revisamos Valores duplicados
print("\nNúmero de filas duplicadas:", spotify_data.duplicated().sum())



Número de filas duplicadas: 0


In [29]:
# Buscamos Anomalías en columnas numéricas
print("\nRevisión de anomalías:")

print("\nPopularity (debe estar entre 0 y 100):\n", spotify_data['popularity'].describe().round(2))

print("\nDanceability (debe estar entre 0 y 1):\n",spotify_data['danceability'].describe().round(2))

print("\nDuration_ms (debe ser positivo y razonable):\n",spotify_data['duration_ms'].describe().round(2))

print("\nTempo (debe ser positivo y razonable):\n",spotify_data['tempo'].describe().round(2))

print("\nLoudness (debe ser negativo y razonable):\n", spotify_data['loudness'].describe().round(2))


Revisión de anomalías:

Popularity (debe estar entre 0 y 100):
 count    1774409.00
mean          76.00
std           15.77
min            0.00
25%           65.00
50%           80.00
75%           88.00
max          100.00
Name: popularity, dtype: float64

Danceability (debe estar entre 0 y 1):
 count    1774409.00
mean           0.68
std            0.14
min            0.00
25%            0.58
50%            0.70
75%            0.78
max            0.99
Name: danceability, dtype: float64

Duration_ms (debe ser positivo y razonable):
 count    1774409.00
mean      193597.34
std        49667.67
min        16320.00
25%       161831.00
50%       185933.00
75%       218423.00
max       939666.00
Name: duration_ms, dtype: float64

Tempo (debe ser positivo y razonable):
 count    1774409.00
mean         122.17
std           28.03
min            0.00
25%          100.02
50%          119.96
75%          140.07
max          236.09
Name: tempo, dtype: float64

Loudness (debe ser negativo y razon

In [31]:
# Filtrar valores atípicos para duration_ms Definir el umbral de 1 hora en milisegundos
umbral_ms = 60 * 60 * 1000  # 3,600,000 ms

# Filtrar eliminando valores mayores a 1 hora
spotify_data = spotify_data[spotify_data['duration_ms'] <= umbral_ms]

# Mostrar el resultado
print("\nDuration_ms (debe ser positivo y razonable):\n",spotify_data['duration_ms'].describe().round(2))


Duration_ms (debe ser positivo y razonable):
 count    1774409.00
mean      193597.34
std        49667.67
min        16320.00
25%       161831.00
50%       185933.00
75%       218423.00
max       939666.00
Name: duration_ms, dtype: float64


In [33]:
# Buscamos inconsistencias en el formato
print("\nRevisión de formatos:")

print("\nSnapshot_date (debe ser fecha):\n",spotify_data['snapshot_date'].head(3))

print("\nAlbum_release_date (debe ser fecha):\n", spotify_data['album_release_date'].head(3))

print("\nIs_explicit (debe ser booleano):\n", spotify_data['is_explicit'].unique())


Revisión de formatos:

Snapshot_date (debe ser fecha):
 0    2025-03-02
1    2025-03-02
2    2025-03-02
Name: snapshot_date, dtype: object

Album_release_date (debe ser fecha):
 0    2024-08-16
1    2024-12-06
2    2024-11-21
Name: album_release_date, dtype: object

Is_explicit (debe ser booleano):
 [False  True]


In [35]:
# Cambiamos el formato de las fechas
spotify_data['snapshot_date'] = pd.to_datetime(spotify_data['snapshot_date'], format='%Y-%m-%d')
spotify_data['album_release_date'] = pd.to_datetime(spotify_data['album_release_date'], format='%Y-%m-%d')

In [37]:
print("\nSnapshot_date (debe ser fecha):\n",spotify_data['snapshot_date'].head(3))
print("\nAlbum_release_date (debe ser fecha):\n",spotify_data['album_release_date'].head(3))


Snapshot_date (debe ser fecha):
 0   2025-03-02
1   2025-03-02
2   2025-03-02
Name: snapshot_date, dtype: datetime64[ns]

Album_release_date (debe ser fecha):
 0   2024-08-16
1   2024-12-06
2   2024-11-21
Name: album_release_date, dtype: datetime64[ns]


In [39]:
# Reemplazamos True and False en la columna 'is_explicit' por valores de 1 and 0 para que sea booleana
spotify_data['is_explicit'] = spotify_data['is_explicit'].astype(int)
spotify_data.head()

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,"Lady Gaga, Bruno Mars",1,0,0,Global,2025-03-02,96,0,...,6,-7.777,0,0.0304,0.3080,0.0000,0.122,0.535,157.969,3
1,4wJ5Qq0jBN4ajy7ouZIV1c,APT.,"ROSÉ, Bruno Mars",2,0,0,Global,2025-03-02,87,0,...,0,-4.477,0,0.2600,0.0283,0.0000,0.355,0.939,149.027,4
2,2CGNAOSuO1MEFCbBRgUzjd,luther (with sza),"Kendrick Lamar, SZA",3,0,0,Global,2025-03-02,89,0,...,2,-7.546,1,0.1250,0.2510,0.0000,0.248,0.576,138.008,4
3,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,Kendrick Lamar,4,0,0,Global,2025-03-02,92,1,...,1,-7.001,1,0.0776,0.0107,0.0000,0.141,0.214,101.061,4
4,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,5,0,0,Global,2025-03-02,94,0,...,2,-10.171,1,0.0358,0.2000,0.0608,0.117,0.438,104.978,4


In [41]:
# Verificar valores únicos en columnas categóricas
print("Key (debe estar entre 0 y 11):\n", spotify_data['key'].unique())

print("\nMode (debe ser 0 o 1):\n", spotify_data['mode'].unique())

print("\nIs_Explicit (debe ser 0 o 1):\n", spotify_data['is_explicit'].unique())

print("\nTime_signature (debe ser un valor común como 3, 4, o 5):\n", spotify_data['time_signature'].unique())

print("\nValores unicos de paises:\n", spotify_data['country'].unique())

Key (debe estar entre 0 y 11):
 [ 6  0  2  1  7 10  8  9 11  4  5  3]

Mode (debe ser 0 o 1):
 [0 1]

Is_Explicit (debe ser 0 o 1):
 [0 1]

Time_signature (debe ser un valor común como 3, 4, o 5):
 [3 4 5 1 0]

Valores unicos de paises:
 ['Global' 'ZA' 'VN' 'VE' 'UY' 'US' 'UA' 'TW' 'TR' 'TH' 'SV' 'SK' 'SG' 'SE'
 'SA' 'RO' 'PY' 'PT' 'PL' 'PK' 'PH' 'PE' 'PA' 'NZ' 'NO' 'NL' 'NI' 'NG'
 'MY' 'MX' 'MA' 'LV' 'LU' 'LT' 'KZ' 'KR' 'JP' 'IT' 'IS' 'IN' 'IL' 'IE'
 'ID' 'HU' 'HN' 'HK' 'GT' 'GR' 'FR' 'FI' 'ES' 'EG' 'EE' 'EC' 'DO' 'DK'
 'DE' 'CZ' 'CR' 'CO' 'CL' 'CH' 'CA' 'BY' 'BR' 'BO' 'BG' 'BE' 'AU' 'AT'
 'AR' 'AE' 'GB']


In [43]:
# Separamos la columna 'artists' en nuevas columnas para diferenciar a los artistas que participan en la cancion
dfartist = spotify_data['artists'].str.split(', ', expand=True)
dfartist.nunique()

0     6871
1     4251
2     1830
3      718
4      323
5      170
6       78
7       35
8       26
9       15
10      11
11       6
12       3
13       2
14       2
15       2
16       2
17       2
18       2
19       2
20       1
21       1
22       1
23       1
24       1
25       1
dtype: int64

In [44]:
# Podemos ver que hay hasta 25 artists en algunas canciones, para facilidad dejaremos unicamente hasta 3 colaboradores en 'feat_artists', divididas en 'feat_1' and 'feat_2' y 'feat_3.
spotify_data = pd.concat([dfartist, spotify_data], axis=1)
spotify_data.rename(columns = {
    0:'main_artist', 
    1:'feat_1', 
    2:'feat_2', 
    3:'feat_3'
    },inplace=True)
spotify_data.sample(5)

,main_artist,feat_1,feat_2,feat_3,4,5,6,7,8,9,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
362239,Chappell Roan,None,None,None,None,None,None,None,None,None,...,11,-5.960,0,0.0356,0.0502,0.000000,0.0881,0.785,116.712,4
569898,Los Ángeles Azules,Maria Becerra,None,None,None,None,None,None,None,None,...,11,-4.012,0,0.0543,0.1660,0.000000,0.0821,0.685,88.002,4
1688756,Hev Abi,None,None,None,None,None,None,None,None,None,...,5,-4.184,0,0.3440,0.2300,0.000003,0.3970,0.524,155.824,4
737229,Jezzy,Leo RD,None,None,None,None,None,None,None,None,...,10,-6.749,0,0.4060,0.0192,0.000002,0.3340,0.889,117.970,4
184508,Kendrick Lamar,Lefty Gunplay,None,None,None,None,None,None,None,None,...,6,-6.679,0,0.2630,0.0837,0.000000,0.4230,0.548,100.036,4


In [46]:
# Y Borramos las columnas restantes hasta 25
spotify_data.drop(spotify_data.iloc[:, 4:26 ], axis=1, inplace=True)
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1774409 entries, 0 to 1775229
Data columns (total 29 columns):
 #   Column              Dtype         
---  ------              -----         
 0   main_artist         object        
 1   feat_1              object        
 2   feat_2              object        
 3   feat_3              object        
 4   spotify_id          object        
 5   name                object        
 6   artists             object        
 7   daily_rank          int64         
 8   daily_movement      int64         
 9   weekly_movement     int64         
 10  country             object        
 11  snapshot_date       datetime64[ns]
 12  popularity          int64         
 13  is_explicit         int32         
 14  duration_ms         int64         
 15  album_name          object        
 16  album_release_date  datetime64[ns]
 17  danceability        float64       
 18  energy              float64       
 19  key                 int64         
 20  loudnes

In [49]:
# borramos la columna artists ya que lo hemos separado en main artists y featured artists
spotify_data.drop(['artists'], axis=1, inplace= True)

In [51]:
# Creamos una nueva columna para tener el Año de lanzamiento
spotify_data['release_year'] = spotify_data['album_release_date'].dt.year

In [53]:
# Creamos una columna con duracion en minutos
spotify_data['duration_min']= round(spotify_data['duration_ms'] / (1000*60), 2)

In [55]:
# Agregamos la nueva columna "key_name" basada en la columna "key"
key_mapping = {
    0: 'C',   # do
    1: 'C#',  # do sostenido
    2: 'D',   # re
    3: 'Eb',  # mi bemol
    4: 'E',   # mi
    5: 'F',   # fa
    6: 'F#',  # fa sostenido
    7: 'G',   # sol
    8: 'G#',  # sol sostenido
    9: 'A',   # la
    10: 'Bb', # si bemol
    11: 'B'   # si
}

spotify_data["key_name"] = spotify_data["key"].map(key_mapping)

# Mostrar las primeras filas para verificar
print(spotify_data[["key", "key_name"]].head())

   key key_name
0    6       F#
1    0        C
2    2        D
3    1       C#
4    2        D


In [57]:
spotify_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1774409 entries, 0 to 1775229
Data columns (total 31 columns):
 #   Column              Dtype         
---  ------              -----         
 0   main_artist         object        
 1   feat_1              object        
 2   feat_2              object        
 3   feat_3              object        
 4   spotify_id          object        
 5   name                object        
 6   daily_rank          int64         
 7   daily_movement      int64         
 8   weekly_movement     int64         
 9   country             object        
 10  snapshot_date       datetime64[ns]
 11  popularity          int64         
 12  is_explicit         int32         
 13  duration_ms         int64         
 14  album_name          object        
 15  album_release_date  datetime64[ns]
 16  danceability        float64       
 17  energy              float64       
 18  key                 int64         
 19  loudness            float64       
 20  mode   

In [59]:
spotify_data.isna().sum()

main_artist                 0
feat_1                1056014
feat_2                1562040
feat_3                1708667
spotify_id                  0
name                        0
daily_rank                  0
daily_movement              0
weekly_movement             0
country                     0
snapshot_date               0
popularity                  0
is_explicit                 0
duration_ms                 0
album_name                  0
album_release_date          0
danceability                0
energy                      0
key                         0
loudness                    0
mode                        0
speechiness                 0
acousticness                0
instrumentalness            0
liveness                    0
valence                     0
tempo                       0
time_signature              0
release_year                0
duration_min                0
key_name                    0
dtype: int64

In [61]:
spotify_data['country'].unique()

array(['Global', 'ZA', 'VN', 'VE', 'UY', 'US', 'UA', 'TW', 'TR', 'TH',
       'SV', 'SK', 'SG', 'SE', 'SA', 'RO', 'PY', 'PT', 'PL', 'PK', 'PH',
       'PE', 'PA', 'NZ', 'NO', 'NL', 'NI', 'NG', 'MY', 'MX', 'MA', 'LV',
       'LU', 'LT', 'KZ', 'KR', 'JP', 'IT', 'IS', 'IN', 'IL', 'IE', 'ID',
       'HU', 'HN', 'HK', 'GT', 'GR', 'FR', 'FI', 'ES', 'EG', 'EE', 'EC',
       'DO', 'DK', 'DE', 'CZ', 'CR', 'CO', 'CL', 'CH', 'CA', 'BY', 'BR',
       'BO', 'BG', 'BE', 'AU', 'AT', 'AR', 'AE', 'GB'], dtype=object)

In [63]:
# Función para obtener el nombre completo del país usando pyCountry
def get_country_name(country_code):
    if isinstance(country_code, str):  # Asegura que es un string
        country = pc.countries.get(alpha_2=country_code.upper())  # Convierte a mayúsculas por si acaso
        if country:
            return country.name
    return 'Global'  # Devuelve Global si no se encuentra el país

In [65]:
get_country_name('MX')

'Mexico'

In [67]:
# Agregar columna 'country_name' al dataframe de Spotify
spotify_data['country_name'] = spotify_data['country'].map(get_country_name)

In [68]:
spotify_data['country_name'].unique()

array(['Global', 'South Africa', 'Viet Nam',
       'Venezuela, Bolivarian Republic of', 'Uruguay', 'United States',
       'Ukraine', 'Taiwan, Province of China', 'Türkiye', 'Thailand',
       'El Salvador', 'Slovakia', 'Singapore', 'Sweden', 'Saudi Arabia',
       'Romania', 'Paraguay', 'Portugal', 'Poland', 'Pakistan',
       'Philippines', 'Peru', 'Panama', 'New Zealand', 'Norway',
       'Netherlands', 'Nicaragua', 'Nigeria', 'Malaysia', 'Mexico',
       'Morocco', 'Latvia', 'Luxembourg', 'Lithuania', 'Kazakhstan',
       'Korea, Republic of', 'Japan', 'Italy', 'Iceland', 'India',
       'Israel', 'Ireland', 'Indonesia', 'Hungary', 'Honduras',
       'Hong Kong', 'Guatemala', 'Greece', 'France', 'Finland', 'Spain',
       'Egypt', 'Estonia', 'Ecuador', 'Dominican Republic', 'Denmark',
       'Germany', 'Czechia', 'Costa Rica', 'Colombia', 'Chile',
       'Switzerland', 'Canada', 'Belarus', 'Brazil',
       'Bolivia, Plurinational State of', 'Bulgaria', 'Belgium',
       'Australia

In [71]:
# Diccionario de continentes (ISO Alpha-2 de países → Continente)
continent_mapping = {
    "AF": "África", 
    "NA": "América del Norte", 
    "SA": "América del Sur",
    "AS": "Asia", 
    "EU": "Europa", 
    "OC": "Oceanía", 
    "AN": "Antártida"
}

# Función para obtener el continente del país usando pycountry_convert
def get_continent(country_code):
    try:
        country_alpha2 = country_code.upper()  # Asegurar formato ISO Alpha-2
        country_continent_code = pyc.country_alpha2_to_continent_code(country_alpha2)
        continent_names = {
            "AF": "África", "NA": "América del Norte", "SA": "América del Sur",
            "AS": "Asia", "EU": "Europa", "OC": "Oceanía", "AN": "Antártida"
        }
        return continent_names.get(country_continent_code, "Desconocido")
    except:
        return "Global"

In [73]:
get_continent('MX')

'América del Norte'

In [75]:
# Agregar columna continente al dataframe de Spotify
spotify_data['continent'] = spotify_data['country'].map(get_continent)

In [77]:
spotify_data['continent'].unique()

array(['Global', 'África', 'Asia', 'América del Sur', 'América del Norte',
       'Europa', 'Oceanía'], dtype=object)

In [79]:
spotify_data['continent'].value_counts()

continent
Europa               702691
Asia                 414741
América del Norte    244328
América del Sur      242557
África                97454
Oceanía               48388
Global                24250
Name: count, dtype: int64

In [81]:
# Creamos una columna con el Ranking de la semana previa
spotify_data['past_week_rank'] = spotify_data['daily_rank'] + spotify_data['weekly_movement']

In [83]:
# Revision final
spotify_data.sample(5)

,main_artist,feat_1,feat_2,feat_3,spotify_id,name,daily_rank,daily_movement,weekly_movement,country,...,liveness,valence,tempo,time_signature,release_year,duration_min,key_name,country_name,continent,past_week_rank
66815,Doechii,None,None,None,1eTaznNW4Xxtx9za2SMTXB,DENIAL IS A RIVER,30,1,3,IS,...,0.0795,0.622,93.292,4,2024,2.66,F#,Iceland,Europa,33
1281709,Grupo Frontera,Grupo Firme,None,None,0O3U5iwTbiXCREMkvotJuN,EL AMOR DE SU VIDA,11,0,-1,GT,...,0.3380,0.746,151.701,4,2023,2.76,A,Guatemala,América del Norte,10
410494,Engel Montaz,Kuv507,Latinnites Music,None,2FFr0vdtqPBTK3RW0c7T5C,Cupido,8,3,0,VE,...,0.2640,0.648,96.041,4,2024,3.58,B,"Venezuela, Bolivarian Republic of",América del Sur,8
816212,Lyodra,None,None,None,3FwQ3RYYEekzjEeL3jdpUZ,Pesan Terakhir,36,7,9,MY,...,0.0940,0.140,120.049,4,2021,4.35,D,Malaysia,Asia,45
1206361,Separ,Peter Pann,None,None,3BQqSkF8Raj9lEzJdscmmu,Bbejbb,1,0,0,SK,...,0.1050,0.240,160.073,4,2024,2.90,D,Slovakia,Europa,1


## Exportamos el archivo con datos limpios

In [86]:
# Guardar el conjunto de datos limpios
spotify_data.to_csv('spotify_clean.csv', index=False)